# 택시 문제
## 뉴욕시의 택시 데이터 셋으로 예측 모델

- 데이터셋 출처: 캐글의 NYC_taxi.csv
- 조건: 뉴욕시 내
- 거리에 따른 요금 예측 => 회귀
- 승차에 따른 하차 지역 예측 => 분류

In [1]:
# 모듈 로딩
import pandas as pd
import numpy as np

In [34]:
# 데이터 로딩
taxi_df=pd.read_csv('train.csv')
print(taxi_df)

: 

: 

- taxi_df에는 55423856행이 있음

In [3]:
# 데이터 로딩(행 50000개만)
taxi_df_50000=pd.read_csv('train.csv',nrows=50000,parse_dates=['pickup_datetime'])
print(taxi_df_50000)

                                 key  fare_amount           pickup_datetime  \
0        2009-06-15 17:26:21.0000001          4.5 2009-06-15 17:26:21+00:00   
1        2010-01-05 16:52:16.0000002         16.9 2010-01-05 16:52:16+00:00   
2       2011-08-18 00:35:00.00000049          5.7 2011-08-18 00:35:00+00:00   
3        2012-04-21 04:30:42.0000001          7.7 2012-04-21 04:30:42+00:00   
4      2010-03-09 07:51:00.000000135          5.3 2010-03-09 07:51:00+00:00   
...                              ...          ...                       ...   
49995    2013-06-12 23:25:15.0000004         15.0 2013-06-12 23:25:15+00:00   
49996    2015-06-22 17:19:18.0000007          7.5 2015-06-22 17:19:18+00:00   
49997   2011-01-30 04:53:00.00000063          6.9 2011-01-30 04:53:00+00:00   
49998   2012-11-06 07:09:00.00000069          4.5 2012-11-06 07:09:00+00:00   
49999    2010-01-13 08:13:14.0000007         10.9 2010-01-13 08:13:14+00:00   

       pickup_longitude  pickup_latitude  dropoff_l

- nrows=옵션을 이용하여 불러들인 행수를 정할 수 있음
- parse_dates=['pickup_datetime']을 쓰면 pickup_datetime열이  
  datetime64[ns, UTC]타입으로 변경됨

In [4]:
# 데이터 확인
taxi_df_50000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                50000 non-null  object             
 1   fare_amount        50000 non-null  float64            
 2   pickup_datetime    50000 non-null  datetime64[ns, UTC]
 3   pickup_longitude   50000 non-null  float64            
 4   pickup_latitude    50000 non-null  float64            
 5   dropoff_longitude  50000 non-null  float64            
 6   dropoff_latitude   50000 non-null  float64            
 7   passenger_count    50000 non-null  int64              
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 3.1+ MB


In [5]:
# 열이름
taxi_df_50000.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [6]:
# 결측치 및 이상치 체크
taxi_df_50000.isnull().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [7]:
# 분포 확인
taxi_df_50000.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,11.364171,-72.509756,39.933759,-72.504616,39.926251,1.667840
std,9.685557,10.393860,6.224857,10.407570,6.014737,1.289195
min,-5.000000,-75.423848,-74.006893,-84.654241,-74.006377,0.000000
25%,6.000000,-73.992062,40.734880,-73.991152,40.734372,1.000000
50%,8.500000,-73.981840,40.752678,-73.980082,40.753372,1.000000
75%,12.500000,-73.967148,40.767360,-73.963584,40.768167,2.000000
max,200.000000,40.783472,401.083332,40.851027,43.415190,6.000000


In [ ]:
np.power((taxi_df_5000['pickup_longitude']-taxi_df_5000['dropoff_longitude']),2)+

0       0.000007
1       0.001353
2       0.000072
3       0.000020
4       0.000131
          ...   
4995    0.000023
4996    0.000022
4997    0.000009
4998    0.000157
4999    0.000353
Length: 5000, dtype: float64

In [ ]:
# 거리(distance)열 추가
taxi_df_5000['distance']=np.power((taxi_df_5000['pickup_longitude']-taxi_df_5000['dropoff_longitude'])**2+(taxi_df_5000['pickup_latitude']-taxi_df_5000['dropoff_latitude'])**2,0.5)
taxi_df_5000

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,0.009436
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,0.079696
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,0.013674
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,0.025340
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,0.019470
...,...,...,...,...,...,...,...,...,...
4995,2010-09-04 12:45:32.0000001,6.5,2010-09-04 12:45:32+00:00,-73.980622,40.745157,-73.975777,40.755863,1,0.011751
4996,2011-06-13 10:53:52.0000006,4.9,2011-06-13 10:53:52+00:00,-73.956664,40.766989,-73.961371,40.772639,1,0.007354
4997,2013-12-12 21:26:40.0000004,7.5,2013-12-12 21:26:40+00:00,-73.988707,40.734343,-73.991737,40.722582,1,0.012145
4998,2010-03-08 14:01:00.00000067,7.7,2010-03-08 14:01:00+00:00,-73.952795,40.776642,-73.965320,40.759412,5,0.021301


In [ ]:
# 새로운 열 추가 되었는지 확인
taxi_df_5000.columns

Index(['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'distance'],
      dtype='object')